Residential MCE Bill (Generation Charges Only)

"EV, Residential Rates for Electric Vehicle Owners" plus "Deep Green (Residential)"

In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [2]:
from cost.utility_rate.models import RatePlan
from load.customer.models import Meter

In [3]:
from datetime import datetime

# Step 1. Choose start and end_limit dates (Jan. 1 2018 through Jan. 31 2018)
start = datetime(2018, 1, 1)
end_limit = datetime(2018, 2, 1)

# Step 2. Get meter data
meter = Meter.objects.get(sa_id=8943913372)
intervalframe = meter.intervalframe.filter_by_datetime(start=start, end_limit=end_limit)

# Step 3. Get rates (OpenEI format)
ev_plan = RatePlan.objects.get(name="EV, Residential Rates for Electric Vehicle Owners")
ev_rates = ev_plan.get_latest_rate_collection(start)
ev_openei_rate_data = ev_rates.openei_rate_data

deep_green_plan = RatePlan.objects.get(name="Deep Green (Residential)")
deep_green_rates = deep_green_plan.get_latest_rate_collection(start)
deep_green_openei_rate_data = deep_green_rates.openei_rate_data

In [4]:
from beo_datastore.libs.bill import ValidationBill

# Step 4. Generate bills
ev_bill = ValidationBill(intervalframe=intervalframe, openei_rate_data=ev_openei_rate_data)
deep_green_bill = ValidationBill(intervalframe=intervalframe, openei_rate_data=deep_green_openei_rate_data)

display(ev_bill.total_dataframe)
display(deep_green_bill.total_dataframe)

,category,description,tou_period,count,count_unit,rate,rate_unit,pro_rata,total
0,energy,Energy Charge - Winter Peak,3,249.44,kwh,0.055,$/kwh,1,13.7192
1,energy,Energy Charge - Winter Part-Peak,4,220.61,kwh,0.03,$/kwh,1,6.6183
2,energy,Energy Charge - Winter Off-Peak,5,1155.38,kwh,0.03,$/kwh,1,34.6614
Total,,,,1625.43,,,,,54.9989


,category,description,tou_period,count,count_unit,rate,rate_unit,pro_rata,total
0,energy,Energy Charge - Deep Green $0.01/kWh,0,1625.43,kwh,0.01,$/kwh,1,16.2543
Total,,,,1625.43,,,,,16.2543


In [5]:
from beo_datastore.libs.bill import BillingCollection

# Step 5. Consolodate bills
bill = BillingCollection(bills=[ev_bill, deep_green_bill])

bill.total_dataframe

,category,description,tou_period,count,count_unit,rate,rate_unit,pro_rata,total
0,energy,Energy Charge - Winter Peak,3,249.44,kwh,0.055,$/kwh,1,13.7192
1,energy,Energy Charge - Winter Part-Peak,4,220.61,kwh,0.03,$/kwh,1,6.6183
2,energy,Energy Charge - Winter Off-Peak,5,1155.38,kwh,0.03,$/kwh,1,34.6614
0,energy,Energy Charge - Deep Green $0.01/kWh,0,1625.43,kwh,0.01,$/kwh,1,16.2543
Total,,,,3250.86,,,,,71.2532
